Y-DATA 2021/21<br/>
Large-scale ML HW3<br/>
Serge Tochilov

In [ ]:
# Checklist:
# AWS emr-5.29.0
# MASTER r5d.8xlarge 1x, no EBS
# CORE r5d.8xlarge 4x, no EBS
# Custom bootstrap action: s3://ydatazian/bootstrap.sh
# Allow ssh in master node security group

> There might be older Numpy versions installed behind the latest, clean that. Add libs missed from `emr-5.29.0` AWS image.

In [ ]:
! sudo pip-3.6 uninstall -y numpy
! sudo pip-3.6 uninstall -y numpy
! sudo pip-3.6 uninstall -y numpy
! sudo pip-3.6 install numpy
! sudo pip-3.6 install pypandoc
! sudo pip-3.6 install pyspark==2.4.8

Uninstalling numpy-1.19.5:
  Successfully uninstalled numpy-1.19.5
You are using pip version 9.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Uninstalling numpy-1.14.5:
  Successfully uninstalled numpy-1.14.5
You are using pip version 9.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Uninstalling numpy-1.14.5:
  Successfully uninstalled numpy-1.14.5
You are using pip version 9.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
You are using pip version 9.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Running setup.py install for pypandoc ... - don

In [ ]:
import tqdm.notebook as tqdm

# SparkSession

https://spark.apache.org/docs/2.4.4/api/python/pyspark.html

https://spark.apache.org/docs/2.4.4/api/python/pyspark.sql.html

In [ ]:
import findspark
findspark.init()
import sys
sys.path.insert(0, "/home/hadoop/ydata_lsml")
import spark_utils
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext("yarn", "My App", conf=spark_utils.get_spark_conf())
se = SparkSession(sc)
spark_utils.print_ui_links()

NameNode: http://ec2-44-230-208-59.us-west-2.compute.amazonaws.com:50070
YARN: http://ec2-44-230-208-59.us-west-2.compute.amazonaws.com:8088
Spark UI: http://ec2-44-230-208-59.us-west-2.compute.amazonaws.com:20888/proxy/application_1623881339968_0001


# Register all tables for SQL queries

In [ ]:
from IPython.display import display
tables = ["clicks_test", "clicks_train",
          "documents_categories", "documents_entities", "documents_meta", "documents_topics",
          "events", "page_views", "page_views_sample", "promoted_content"]
for name in tqdm.tqdm(tables):
    df = se.read.parquet("s3://ydatazian/{}.parquet".format(name))
    df.registerTempTable(name)
    print(name)
    display(df.limit(3).toPandas())

  0%|          | 0/10 [00:00<?, ?it/s]

clicks_test


,display_id,ad_id
0,16874594,66758
1,16874594,150083
2,16874594,162754


clicks_train


,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1


...

,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1


page_views_sample


,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1


promoted_content


,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7


In [ ]:
display(se.sql("select count(*) from clicks_test").toPandas())

,count(1)
0,32225162


# Download Vowpal Wabbit

In [ ]:
! wget http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624 -O ./vw
! chmod +x ./vw
! ./vw --version

--2021-06-16 22:13:35--  http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624
Resolving finance.yendor.com (finance.yendor.com)... 69.163.152.190
Connecting to finance.yendor.com (finance.yendor.com)|69.163.152.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9467376 (9.0M)
Saving to: ‘./vw’

./vw                100%[===================>]   9.03M  19.5MB/s    in 0.5s    

2021-06-16 22:13:35 (19.5 MB/s) - ‘./vw’ saved [9467376/9467376]

8.7.0


# Prepare dataset for VW

We will predict a *click* based on:
- ad_id
- document_id
- campaign_id
- advertiser_id

In [ ]:
%%time
se.sql("""
select 
    clicks_train.clicked,
    clicks_train.display_id,
    clicks_train.ad_id,
    promoted_content.document_id,
    promoted_content.campaign_id,
    promoted_content.advertiser_id
from
    clicks_train
join
    promoted_content
on
    clicks_train.ad_id = promoted_content.ad_id
""").write.parquet("/train_features.parquet", mode='overwrite')

CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 58.1 s


In [ ]:
se.read.parquet("/train_features.parquet").show(5)

+-------+----------+------+-----------+-----------+-------------+
|clicked|display_id| ad_id|document_id|campaign_id|advertiser_id|
+-------+----------+------+-----------+-----------+-------------+
|      0|         1| 42337|     938164|       5969|         1499|
|      0|         1|139684|    1085937|      17527|         2563|
|      1|         1|144739|    1337362|      18488|         2909|
|      0|         1|156824|     992370|       7283|         1919|
|      0|         1|279295|    1670176|      27524|         1820|
+-------+----------+------+-----------+-----------+-------------+
only showing top 5 rows



In [ ]:
# Format: [Label] [Importance] [Base] [Tag]|Namespace Features |Namespace Features ... |Namespace Features
# https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format
def vw_row_mapper(row):
    clicked = None
    features = []
    for k, v in row.asDict().items():
        if k == 'clicked':
            clicked = '1' if v == '1' else '-1'
        else:
            features.append(k + "_" + v)
    tag = row.display_id + "_" + row.ad_id
    return "{} {}| {}".format(clicked, tag, " ".join(features))

r = se.read.parquet("/train_features.parquet").take(1)[0]
print(r)
print(vw_row_mapper(r))

Row(clicked='0', display_id='1', ad_id='42337', document_id='938164', campaign_id='5969', advertiser_id='1499')
-1 1_42337| display_id_1 ad_id_42337 document_id_938164 campaign_id_5969 advertiser_id_1499


In [ ]:
%%time
! hdfs dfs -rm -r /train_features.txt
(
    se.read.parquet("/train_features.parquet")
    .rdd
    .map(vw_row_mapper)
    .saveAsTextFile("/train_features.txt")
)

rm: `/train_features.txt': No such file or directory
CPU times: user 49.3 ms, sys: 39.3 ms, total: 88.7 ms
Wall time: 3min 42s


In [ ]:
# copy file to local master node
! rm /mnt/train.txt
! hdfs dfs -getmerge /train_features.txt /mnt/train.txt
# preview local file
! head -n 5 /mnt/train.txt

rm: cannot remove ‘/mnt/train.txt’: No such file or directory
-1 1_42337| display_id_1 ad_id_42337 document_id_938164 campaign_id_5969 advertiser_id_1499
-1 1_139684| display_id_1 ad_id_139684 document_id_1085937 campaign_id_17527 advertiser_id_2563
1 1_144739| display_id_1 ad_id_144739 document_id_1337362 campaign_id_18488 advertiser_id_2909
-1 1_156824| display_id_1 ad_id_156824 document_id_992370 campaign_id_7283 advertiser_id_1919
-1 1_279295| display_id_1 ad_id_279295 document_id_1670176 campaign_id_27524 advertiser_id_1820


# Train VW
https://vowpalwabbit.org/tutorials/getting_started.html

https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments

In [ ]:
! ./vw -d /mnt/train.txt -b 24 -c -k --ftrl --passes 1 -f model --holdout_off --loss_function logistic --random_seed 42 --progress 8000000

final_regressor = model
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = /mnt/train.txt.cache
Reading datafile = /mnt/train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.460142 0.460142      8000000      8000000.0  -1.0000  -1.2983        6
0.456159 0.452175     16000000     16000000.0  -1.0000  -1.4733        6
0.453326 0.447661     24000000     24000000.0  -1.0000  -2.3174        6
0.452628 0.450533     32000000     32000000.0   1.0000  -0.5719        6
0.452887 0.453924     40000000     40000000.0  -1.0000  -1.2648        6
0.452010 0.447625     48000000     48000000.0  -1.0000  -2.6227        6
0.451328 0.447232     56000000     56000000.0  -1.0000  -3.4451        6
0.450649 0.445902     64000000     64000000.0  -1.0000  -1.

In [ ]:
# make prediction with VW
! echo "? tag1| ad_id_144739 document_id_1337362 campaign_id_18488 advertiser_id_2909" > /mnt/test.txt
! echo "? tag2| ad_id_156824 document_id_992370 campaign_id_7283 advertiser_id_1919" >> /mnt/test.txt
! ./vw -d /mnt/test.txt -i model -t -k -p /mnt/predictions.txt --progress 1000000 --link=logistic
# predicted probabilities of "1" class
! cat /mnt/predictions.txt

only testing
predictions = /mnt/predictions.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /mnt/test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features

finished run
number of examples = 2
weighted example sum = 2.000000
weighted label sum = 0.000000
average loss = 5.674478
total feature number = 10
0.319262 tag1
0.036172 tag2


# HW 3. Baseline model

Train a baseline model (**any model** you can think of, not necessarily VW) using the following features (features will bring you **0.5 points each**, you can think of your own features, **maximum 8 points** for this task):
- **clicked**
- geo_location features (country, state, dma)
- day_of_week (from timestamp, use `date.isoweekday()`)
- ad_id
- campaign_id
- advertiser_id
- ad_document_id
- display_document_id
- platform

Make submission to Kaggle to know your leaderboard score.

If you want to create a dev set, make a 90%/10% split of training data by `display_id`.

> To be able to construct the required features, including compound `ad_id/document_id` and `display_id/document_id`, as well as additional features, we employed SparkSQL and select all the data of interest in one SQL statement. We equipped it with the test/train format switch to avoid boilerplating. The statement joins `clicks_...` with `promoted_content` on `ad_id`, `events` on `display_id`, and `documents_*` through `events` on `display_id` and `document_id`. In addition to the required we add the following features:

> - `uuid`
- plain `display_id`
- plain `document_id`
- `topic_id`
- `documents_topics.confidence_level`
- `source_id`
- `publisher_id`
- `publish_time` (date and time)
- `category_id`
- `documents_categories.confidence_level`

> The maximum number of `current features` reported by VW was $22$.

> The values were stripped of non-alphanumeric characters, and their aliases were selected to be as short as possible (one letter in most cases) which significantly saved disk space allocated by the map-reduced text files. For example, `train_features_hw3.txt`'s size decreased from ~220G to ~100G comparing to the one with the default column names, and the latter fit to the suggested `r5d.8xlarge`-based Hadoop configuration.

> We employed the suggested WV hyperparameters with one train pass as more passes didn't improve the loss. Total running time from the start until the Kaggle submitting was about 2.5h. The map-reduce train stage were the longest (~1h).

> The latest submission (`sergetochilov`, `hw3-11`) has 0.64383/0.64360 private/public score (260th place).

In [ ]:
sql_q = """
select
    {target}clicked,
    clicks_{data}.display_id as d,
    clicks_{data}.ad_id as a,
    promoted_content.campaign_id as c, 
    promoted_content.advertiser_id as v,
    events.uuid as u,
    events.document_id as o,
    events.timestamp,
    events.platform as p,
    events.geo_location,
    documents_topics.topic_id as t,
    documents_topics.confidence_level,
    documents_meta.source_id as s,
    documents_meta.publisher_id as p,
    documents_meta.publish_time as b,
    documents_categories.category_id as e,
    documents_categories.confidence_level as n
from
    clicks_{data}
left join
    promoted_content
    on
    clicks_{data}.ad_id = promoted_content.ad_id
left join
    events
    on
    clicks_{data}.display_id = events.display_id
left join
    documents_topics
    on
    clicks_{data}.display_id = events.display_id
    and
    events.document_id = documents_topics.document_id
left join
    documents_meta
    on
    clicks_{data}.display_id = events.display_id
    and
    events.document_id = documents_meta.document_id
left join
    documents_categories
    on
    clicks_{data}.display_id = events.display_id
    and
    events.document_id = documents_categories.document_id
"""

In [ ]:
%%time
se.sql(sql_q.format(target='clicks_train.', data='train')).write.parquet("/train_features_hw3.parquet", mode='overwrite')
se.read.parquet("/train_features_hw3.parquet").show(5)

+-------+-------+-----+----+----+--------------+-------+---------+---+------------+---+------------------+---+---+-------------------+----+----+
|clicked|      d|    a|   c|   v|             u|      o|timestamp|  p|geo_location|  t|  confidence_level|  s|  b|                  l|   e|   n|
+-------+-------+-----+----+----+--------------+-------+---------+---+------------+---+------------------+---+---+-------------------+----+----+
|      0|3436070|36107|2956|1445|55e1db49ff4eef|1000240|223783698|  1|   US>CA>803|138|0.0894683491081469|890|522|2015-12-22 08:00:00|1505|0.92|
|      0|3436070|36107|2956|1445|55e1db49ff4eef|1000240|223783698|  1|   US>CA>803|138|0.0894683491081469|890|522|2015-12-22 08:00:00|1503|0.07|
|      0|3436070|36107|2956|1445|55e1db49ff4eef|1000240|223783698|  1|   US>CA>803|143|0.0881824742656303|890|522|2015-12-22 08:00:00|1505|0.92|
|      0|3436070|36107|2956|1445|55e1db49ff4eef|1000240|223783698|  1|   US>CA>803|143|0.0881824742656303|890|522|2015-12-22 08:00

In [ ]:
from datetime import date

def vw_row_mapper_hw3(row):
    clicked = None
    features = []

    for k, v in row.asDict().items():
        if k == 'clicked':
            clicked = '1' if v == '1' else '-1'
        elif k == 'timestamp':
            features += [f"dw{date.fromtimestamp(int(v)).isoweekday()}"]
        elif k == 'geo_location':
            geos = [] if v is None else v.split('>')
            geos += [' '] * max(3 - len(geos), 0)
            features += [f"{geo}{loc.lower()}" for geo, loc in zip(["x", "y", "z"], geos[:3])]
        elif k == 'publish_time':
            features += [] if v is None else v.split(' ')
        elif k == 'confidence_level':
            features += [] if v is None else [f"f{float(v)*10:0.2f}"]
        else:
            features += [f"{k}{v}"]

    features += [f"{col}o{row[col]}{row.o}" for col in ["d", "a"]]
    features = ' '.join(features).replace('.', '').replace(':', '').replace('-', '')
    tag = row.d + "_" + row.a
    return f"{clicked} {tag}| {features}"

r = se.read.parquet("/train_features_hw3.parquet").take(1)[0]
print(r)
print(vw_row_mapper_hw3(r))

Row(clicked='0', d='3436070', a='36107', c='2956', v='1445', u='55e1db49ff4eef', o='1000240', timestamp='223783698', p='1', geo_location='US>CA>803', t='138', confidence_level='0.0894683491081469', s='890', b='522', l='2015-12-22 08:00:00', e='1505', n='0.92')
-1 3436070_36107| d3436070 a36107 c2956 v1445 u55e1db49ff4eef o1000240 dw4 p1 xus yca z803 t138 f089 s890 b522 20151222 080000 e1505 n092 do34360701000240 ao361071000240


In [ ]:
%%time
! hdfs dfs -rm -r /train_features_hw3.txt
(
    se.read.parquet("/train_features_hw3.parquet")
    .rdd
    .map(vw_row_mapper_hw3)
    .saveAsTextFile("/train_features_hw3.txt")
)

rm: `/train_features_hw3.txt': No such file or directory
CPU times: user 354 ms, sys: 59.8 ms, total: 414 ms
Wall time: 53min 42s


In [ ]:
%%time
# copy file to local master node
! rm /mnt/train_hw3.txt
! hdfs dfs -getmerge /train_features_hw3.txt /mnt/train_hw3.txt
! head -n 5 /mnt/train_hw3.txt
! ls -la /mnt/train_hw3.txt
! wc -l /mnt/train_hw3.txt

rm: cannot remove ‘/mnt/train_hw3.txt’: No such file or directory
-1 3436070_36107| d3436070 a36107 c2956 v1445 u55e1db49ff4eef o1000240 dw4 p1 xus yca z803 t138 f089 s890 b522 20151222 080000 e1505 n092 do34360701000240 ao361071000240
-1 3436070_36107| d3436070 a36107 c2956 v1445 u55e1db49ff4eef o1000240 dw4 p1 xus yca z803 t138 f089 s890 b522 20151222 080000 e1503 n007 do34360701000240 ao361071000240
-1 3436070_36107| d3436070 a36107 c2956 v1445 u55e1db49ff4eef o1000240 dw4 p1 xus yca z803 t143 f088 s890 b522 20151222 080000 e1505 n092 do34360701000240 ao361071000240
-1 3436070_36107| d3436070 a36107 c2956 v1445 u55e1db49ff4eef o1000240 dw4 p1 xus yca z803 t143 f088 s890 b522 20151222 080000 e1503 n007 do34360701000240 ao361071000240
-1 3436070_36107| d3436070 a36107 c2956 v1445 u55e1db49ff4eef o1000240 dw4 p1 xus yca z803 t89 f068 s890 b522 20151222 080000 e1505 n092 do34360701000240 ao361071000240
-rw-r--r-- 1 hadoop hadoop 107581705848 Jun 16 23:28 /mnt/train_hw3.txt
623822931 /mn

In [ ]:
%%time
! ./vw -d /mnt/train_hw3.txt -b 24 -c -k --ftrl --passes 1 -f model_hw3.vw.bin --holdout_off --loss_function logistic --random_seed 42 --progress 8000000

final_regressor = model_hw3.vw.bin
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = /mnt/train_hw3.txt.cache
Reading datafile = /mnt/train_hw3.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.455525 0.455525      8000000      8000000.0  -1.0000  -1.0349       22
0.449667 0.443809     16000000     16000000.0   1.0000  -0.4090       21
0.449558 0.449340     24000000     24000000.0  -1.0000  -1.0984       22
0.448532 0.445454     32000000     32000000.0  -1.0000  -1.4530       22
0.445824 0.434993     40000000     40000000.0  -1.0000  -1.8100       22
0.445872 0.446111     48000000     48000000.0  -1.0000  -0.8217       22
0.445176 0.441002     56000000     56000000.0  -1.0000  -1.7705       22
0.443829 0.434399     64000000     64000

# Submitting to Kaggle

Obtain Kaggle API token: https://github.com/Kaggle/kaggle-api#api-credentials

Making a submission: https://github.com/Kaggle/kaggle-api#submit-to-a-competition

In [ ]:
%%time
se.sql(sql_q.format(target='"0" as ', data='test')).write.parquet("/test_features_hw3.parquet", mode='overwrite')
se.read.parquet("/test_features_hw3.parquet").show(5)

+-------+--------+-----+----+----+--------------+-------+----------+---+------------+---+------------------+-----+---+-------------------+----+----+
|clicked|       d|    a|   c|   v|             u|      o| timestamp|  p|geo_location|  t|  confidence_level|    s|  b|                  l|   e|   n|
+-------+--------+-----+----+----+--------------+-------+----------+---+------------+---+------------------+-----+---+-------------------+----+----+
|      0|20285030|68743|5043|1726|f3e881f6f85843|1000495|1154557712|  1|       NL>11|174| 0.068081067406863|10199|  9|2015-12-22 09:00:00|1607|0.92|
|      0|20285030|68743|5043|1726|f3e881f6f85843|1000495|1154557712|  1|       NL>11|174| 0.068081067406863|10199|  9|2015-12-22 09:00:00|1100|0.07|
|      0|20285030|68743|5043|1726|f3e881f6f85843|1000495|1154557712|  1|       NL>11| 26|0.0661115173841483|10199|  9|2015-12-22 09:00:00|1607|0.92|
|      0|20285030|68743|5043|1726|f3e881f6f85843|1000495|1154557712|  1|       NL>11| 26|0.066111517384148

In [ ]:
%%time
! hdfs dfs -rm -r /test_features_hw3.txt
(
    se.read.parquet("/test_features_hw3.parquet")
    .rdd
    .map(vw_row_mapper_hw3)
    .saveAsTextFile("/test_features_hw3.txt")
)

rm: `/test_features_hw3.txt': No such file or directory
CPU times: user 138 ms, sys: 46.1 ms, total: 184 ms
Wall time: 19min 10s


In [ ]:
%%time
# copy file to local master node
! rm /mnt/test_hw3.txt
! hdfs dfs -getmerge /test_features_hw3.txt /mnt/test_hw3.txt
! head -n 5 /mnt/test_hw3.txt
! ls -la /mnt/test_hw3.txt
! wc -l /mnt/test_hw3.txt

rm: cannot remove ‘/mnt/test_hw3.txt’: No such file or directory
-1 20285030_68743| d20285030 a68743 c5043 v1726 uf3e881f6f85843 o1000495 dw3 p1 xnl y11 z  t174 f068 s10199 b9 20151222 090000 e1607 n092 do202850301000495 ao687431000495
-1 20285030_68743| d20285030 a68743 c5043 v1726 uf3e881f6f85843 o1000495 dw3 p1 xnl y11 z  t174 f068 s10199 b9 20151222 090000 e1100 n007 do202850301000495 ao687431000495
-1 20285030_68743| d20285030 a68743 c5043 v1726 uf3e881f6f85843 o1000495 dw3 p1 xnl y11 z  t26 f066 s10199 b9 20151222 090000 e1607 n092 do202850301000495 ao687431000495
-1 20285030_68743| d20285030 a68743 c5043 v1726 uf3e881f6f85843 o1000495 dw3 p1 xnl y11 z  t26 f066 s10199 b9 20151222 090000 e1100 n007 do202850301000495 ao687431000495
-1 20285030_68743| d20285030 a68743 c5043 v1726 uf3e881f6f85843 o1000495 dw3 p1 xnl y11 z  t250 f057 s10199 b9 20151222 090000 e1607 n092 do202850301000495 ao687431000495
-rw-r--r-- 1 hadoop hadoop 39327884725 Jun 17 00:20 /mnt/test_hw3.txt
225018738 /m

In [ ]:
%%time
! ./vw -d /mnt/test_hw3.txt -i model_hw3.vw.bin -t -k -p /mnt/predictions_hw3.txt --progress 1000000 --link=logistic
# predicted probabilities of "1" class
! head -n 5 /mnt/predictions_hw3.txt

only testing
predictions = /mnt/predictions_hw3.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /mnt/test_hw3.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.031190 1.031190      1000000      1000000.0  -1.0000   0.1530       22
1.012853 0.994515      2000000      2000000.0  -1.0000   0.1170       22
0.989055 0.941460      3000000      3000000.0  -1.0000   0.1228       22
1.118867 1.508303      4000000      4000000.0  -1.0000   0.1468       22
1.131043 1.179747      5000000      5000000.0  -1.0000   0.2423       19
1.130616 1.128482      6000000      6000000.0  -1.0000   0.2540       22
1.145939 1.237881      7000000      7000000.0  -1.0000   0.0599       22
1.204096 1.611189      8000000      8000000.0  -1.0000  

In [ ]:
! wc -l /mnt/predictions_hw3.txt

225018738 /mnt/predictions_hw3.txt


In [ ]:
%%time
from collections import defaultdict

scores_by_display_id_hw3 = defaultdict(dict)
for line in tqdm.tqdm(open('/mnt/predictions_hw3.txt')):
    score, tag = line.strip().split(" ")
    score = float(score)
    display_id, ad_id = tag.split("_")
    scores_by_display_id_hw3[display_id][ad_id] = score

0it [00:00, ?it/s]

CPU times: user 5min 36s, sys: 6.21 s, total: 5min 42s
Wall time: 5min 38s


In [ ]:
len(scores_by_display_id_hw3)
list(scores_by_display_id_hw3.items())[:5]

[('20285030',
  {'68743': 0.146568,
   '141476': 0.131702,
   '204432': 0.227406,
   '243487': 0.164189,
   '243505': 0.17711,
   '326357': 0.305496}),
 ('18865607',
  {'82850': 0.088199,
   '167280': 0.156179,
   '182579': 0.147056,
   '329410': 0.286843}),
 ('17620962',
  {'268690': 0.234892,
   '281546': 0.165246,
   '364186': 0.191365,
   '367652': 0.029157}),
 ('17643187',
  {'20540': 0.196004,
   '155809': 0.218139,
   '160763': 0.210033,
   '209187': 0.235447}),
 ('18057588',
  {'42387': 0.209016,
   '209186': 0.201258,
   '330515': 0.31331,
   '404726': 0.274843})]

In [ ]:
with open("submission_hw3.txt", "w") as f:
    f.write("display_id,ad_id\n")
    for k, vs in tqdm.tqdm_notebook(scores_by_display_id_hw3.items()):
        f.write("{},{}\n".format(
            k,
            " ".join([v[0] for v in sorted(vs.items(), key=lambda x: -x[1])])
        ))

  0%|          | 0/6245533 [00:00<?, ?it/s]

In [ ]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"sergetochilov","key":"f9d0d0849f6ac52abb8ac61befabf518"}' > ~/.kaggle/kaggle.json
! cat ~/.kaggle/kaggle.json
! chmod 600 /home/hadoop/.kaggle/kaggle.json

{"username":"sergetochilov","key":"f9d0d0849f6ac52abb8ac61befabf518"}


In [ ]:
! kaggle competitions submit -f submission_hw3.txt outbrain-click-prediction -m "hw3-11"

100%|████████████████████████████████████████| 260M/260M [00:06<00:00, 41.3MB/s]
Successfully submitted to Outbrain Click Prediction